# 本日の概要
- 本日話すこと
    - J-Quants APIの利用の仕方
    - J-Quants APIを利用した簡単なデータ分析

- 本日話さないこと
    - ゴリゴリのデータ分析
    - データ分析を利用した投資・投資手法の紹介等


プログラミング初心者の方や、これからデータ分析を始める方向けに、  
コードも交えてJ-Quants APIの利用方法について紹介させていただきます！！

# J-QuantsAPIについて

## J-Quants APIの利用の仕方



1. [APIサービス登録ページ](https://jpx-jquants.com/auth/signup)よりJ-Quants APIサービスへ登録

2. 登録後、[ログインページ](https://jpx-jquants.com/auth/signin)から登録したメールアドレス及びパスワードでJ-Quants APIサービスへログインしリフレッシュトークンを取得（取得ボタンを押下すると、自動的にクリップボードへコピーされます）
3. CurlコマンドやPython等を用いて、リフレッシュトークンを利用し、ID tokenを取得
4. CurlコマンドやPython等を用いて、取得したID tokenを利用し、データを取得

なお、J-Quants APIの各種APIの仕様は[こちら](https://jpx.gitbook.io/j-quants-ja/api-reference)をご参照ください。

In [ ]:
#@title 上記3. について、pythonによるID token取得サンプル
import requests
import json

REFRESH_TOKEN = "こちらにクリップボードにコピーしたリフレッシュトークンを貼り付け"
idToken = requests.post(
    f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}"
).json()["idToken"]


In [ ]:
#@title 上記4.について、 pythonによるデータ取得サンプル（銘柄情報）
# 上記３.idTokenを取得していることが前提です
headers = {'Authorization': 'Bearer {}'.format(idToken)}
r = requests.get("https://api.jquants.com/v1/listed/info", headers=headers)

# APIからのresponseはjson形式
info_data = r.json()["info"]

# 取得したデータを5つ表示
info_data[:5]


## J-Quants API各種クライアントライブラリ

J-Quants APIにはより使いやすくなるように、クライアントライブラリをalpacaさんや有志の方に作製いただいております！  
作製いただいた方ありがとうございます！！



* Python用クライアントライブラリ（ [jquants-api-client-python](https://github.com/J-Quants/jquants-api-client-python) ） (有償版対応済)

* R用クライアントライブラリ（ [JQuantsR](https://github.com/J-Quants/JQuantsR) ） (有償版対応済)

今回はPythonを用いるため、Python用クライアントライブラリを利用します。

In [ ]:
# Google Colabでjquants-api-client-pythonを使えるようにインストール
!pip install jquants-api-client


In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')


Driveをマウント後、[こちら](https://github.com/J-Quants/jquants-api-client-python#%E8%A8%AD%E5%AE%9A)を参考にAPI利用のための設定ファイル（jquants-api.toml）を作成します。

この設定を行うことで、jquantsapiパッケージを利用し、リフレッシュトークン及びtokenIDを自動で取得してくれます！！

In [ ]:
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings

import jquantsapi

# pandasデータフレームの描画設定
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 40)
pd.set_option("display.max_colwidth", 80)

warnings.simplefilter('ignore')

# 今回はID及びPWは上記に記載の方法により別ファイルに保存しているため、引数特になしでOK。
cli = jquantsapi.Client()

# 以下のように直接引数として渡してもOKですが、メールアドレスとパスワードが平文でnotebookに残るため、別で設定ファイルを作成したほうが安全です。
# cli = jquantsapi.Client(mail_address="ご登録いただいたメールアドレス", password="ご登録いただいたパスワード")


In [ ]:
# 試しに銘柄情報を取得
cli.get_listed_info()

その他、jquants-api-clientで利用できるものは[こちらの説明](https://github.com/J-Quants/jquants-api-client-python#%E5%AF%BE%E5%BF%9Capi)をご参照ください。


# 銘柄一覧情報 API（毎営業日の21時頃データ更新）

銘柄情報は、東京証券取引所へ上場している銘柄について提供しています。

In [ ]:
df_info = cli.get_listed_info()

# データフレームを確認
df_info.info()

print("\n")

# データフレームを描画
df_info


In [ ]:
# 欠損値の確認
df_info[df_info.isnull().any(axis=1)]


### ひとくちメモ集

- 株式銘柄コードは、(発行体ごとに付番される固有名コード4桁)及び(株券の種類ごとに付番される予備コード1桁)で構成されている。  
（ex.普通株は末尾が0、優先株は末尾が5）


In [ ]:
# 普通株と優先株
df_info[df_info["Code"].isin(["25930", "25935"])]


なお、現在株式銘柄コードは数値のみですが、2024年1月1日以降に新たに設定するコード（新規上場等）から数字のほかに英文字が追加される予定とのことです。  
ご興味ある方は[こちら](https://www.jpx.co.jp/sicc/securities-code/index.html)に資料等があります。

- なお、Sector17Codeが99、またはSecter33Codeが9999のものは、ETFやREIT、インフラファンド等の銘柄になっています。

In [ ]:
df_info[df_info["Sector33Code"] == "9999"]


## 若干のデータ分析

In [ ]:
# 銘柄一覧情報を取得
df_info = cli.get_list()


### 17業種別の企業数

In [ ]:
# ETF, REIT, インフラファンド等の17業種以外のデータを除外
df = df_info[df_info["Sector17Code"] != "99"]

# 17業種名でグループ化
df_sector = df.groupby("Sector17CodeName").size().to_frame(
    "Count").reset_index().sort_values(by="Count", ascending=False)
px.bar(df_sector, x="Sector17CodeName", y="Count", title="17業種別の企業数")


### 33業種別の企業数

In [ ]:
# ETF, REIT, インフラファンド等の33業種以外のデータを除外
df = df_info[df_info["Sector33Code"] != "9999"]

# 33業種名でグループ化
df_sector = df.groupby("Sector33CodeName").size().to_frame(
    "Count").reset_index().sort_values(by="Count", ascending=False)
px.bar(df_sector, x="Sector33CodeName", y="Count", title="33業種別の企業数")


### 市場区分別の企業数

In [ ]:
# 市場区分名でグループ化
df_market = df_info.groupby("MarketCodeName").size().to_frame(
    "Count").reset_index().sort_values(by="Count", ascending=False)
px.bar(df_market, x="MarketCodeName", y="Count", title="市場区分別の企業数")


### 上記分析の両方を加味

In [ ]:
# ETF, REIT, インフラファンド等の33業種以外のデータを除外
df = df_info[df_info["Sector33Code"] != "9999"]

# さらにプライム、スタンダード、グロース及びTokyoProMarket銘柄にフィルタ
df = df[df["MarketCodeName"].isin(["プライム", "スタンダード", "グロース", "TOKYO PRO MARKET"])]

# 業種別及び市場名でグルーピング
df_sector_market = df.groupby([
    "Sector33CodeName", "MarketCodeName"
]).size().to_frame("Count").reset_index().sort_values(by=["MarketCodeName", "Count"],
                                                      ascending=False)

px.bar(df_sector_market,
       x="Sector33CodeName",
       y="Count",
       color="MarketCodeName",
       barmode="relative",
       title="33業種及び市場区分別の企業数")


### 過去時点（１年前）の銘柄一覧で上記同様のグラフ描画

In [ ]:
d = datetime.now() - timedelta(days=365)
df_info = cli.get_listed_info(date_yyyymmdd=d.strftime(format="%Y%m%d"))

# ETF, REIT, インフラファンド等の33業種以外のデータを除外
df = df_info[df_info["Sector33Code"] != "9999"]

# さらにプライム、スタンダード、グロース及びTokyoProMarket銘柄にフィルタ
df = df[df["MarketCodeName"].isin(["プライム", "スタンダード", "グロース", "TOKYO PRO MARKET"])]

# 業種別及び市場名でグルーピング
df_sector_market = df.groupby([
    "Sector33CodeName", "MarketCodeName"
]).size().to_frame("Count").reset_index().sort_values(by=["MarketCodeName", "Count"],
                                                      ascending=False)
px.bar(df_sector_market,
       x="Sector33CodeName",
       y="Count",
       color="MarketCodeName",
       barmode="relative",
       title="33業種及び市場区分別の企業数")


# 株価情報 API（毎営業日の17時00分頃データ更新）

株価データを取得することができます。  
株価及び取引高は、株式分割・併合を考慮した調整済み株価（小数点第２位四捨五入）と調整前の株価の両方を取得することができます。  

なお、データを取得する際には、株式銘柄コード（code）または日付（date）の指定が必須となっています。

また、地取に単独上場している銘柄については、株価配信していません。

### 株式銘柄コード指定Ver

In [ ]:
df_quotes_bycode = cli.get_prices_daily_quotes(code=8697)

# データフレームの情報を取得
df_quotes_bycode.info()

print("\n")

# データフレームを描画
df_quotes_bycode


In [ ]:
# 欠損値の確認
df_quotes_bycode[df_quotes_bycode.isnull().any(axis=1)]


### 取引日指定Ver

In [ ]:
d = datetime.now() - timedelta(days=365)
df_quotes_bydate = cli.get_prices_daily_quotes(date_yyyymmdd=d.strftime(format="%Y%m%d"))

# データフレームの情報を取得
df_quotes_bydate.info()

print("\n")

# データフレームを描画
df_quotes_bydate


In [ ]:
# 欠損値の確認
df_quotes_bydate[df_quotes_bydate.isnull().any(axis=1)]


その日取引が成立しなかった銘柄については、NaNが入っている。

### ひとくちメモ集

- jquants-api-clientでは、`get_price_range()`を使うことで、銘柄指定せずに`from`, `to`で株価データが取得できます！（ありがたい...！）

In [ ]:
d_from = datetime.now() - timedelta(days=365)
d_to = datetime.now() - timedelta(days=335)
df_quotes = cli.get_price_range(start_dt=d_from.strftime(format="%Y%m%d"), end_dt=d_to.strftime(format="%Y%m%d"))
df_quotes


## 若干のデータ分析

- 株価分析でよくある図を作成してみる

In [ ]:
# JPXの株価を取得
df_jpx = cli.get_prices_daily_quotes(code=8697)

# グラフに第2軸を設定
fig = make_subplots(specs=[[{"secondary_y": True}]])

# ローソク足を描画
fig.add_trace(
    go.Candlestick(x=df_jpx.Date,
                   open=df_jpx.AdjustmentOpen,
                   high=df_jpx.AdjustmentHigh,
                   low=df_jpx.AdjustmentLow,
                   close=df_jpx.AdjustmentClose,
                   name="OHLC"))

# 25日移動平均
fig.add_trace(
    go.Scatter(x=df_jpx.Date,
               y=df_jpx.AdjustmentClose.rolling(25).mean(),
               name="25日移動平均線"))

# 75日移動平均
fig.add_trace(
    go.Scatter(x=df_jpx.Date,
               y=df_jpx.AdjustmentClose.rolling(75).mean(),
               name="75日移動平均線"))

# 取引高を第2軸に設定
fig.add_trace(go.Bar(x=df_jpx.Date, y=df_jpx.AdjustmentVolume, name="調整済み取引高"),
              secondary_y=True)
fig.update_layout(title_text="JPX（8697）のローソク足及び出来高")
fig.show()


# 財務情報 API（18時ごろ及び24時30分頃更新）

財務情報は上場会社の四半期及び通期の決算短信に係る情報を取得できます。

In [ ]:
df_fins = cli.get_fins_statements(code=8697)

# データフレームを確認
df_fins.info()

print("\n")

# データフレームを描画
df_fins


In [ ]:
# 欠損値の確認
df_fins[df_fins.isnull().any(axis=1)]


取得したデータフレームには空欄の箇所が多くみられるが、Null値は存在しない。  
本APIのデータは、各企業が適時開示したXBRLデータに基づき作成しており、当該XBRLデータにおいて、データの無い箇所には空文字が入っていることから、本APIにおいても空欄となっている箇所が多くある。

## 若干のデータ分析

In [ ]:
# 過去の一定期間の財務情報を取得
d_from = datetime.now() - timedelta(days=300)
d_to = datetime.now() - timedelta(days=270)
df_fins = cli.get_statements_range(start_dt=d_from.strftime(format="%Y%m%d"), end_dt=d_to.strftime(format="%Y%m%d"))


In [ ]:
# 書類種別の項目を表示
df_fins["TypeOfDocument"].value_counts()


書類種別には、連結（Consolidated）や非連結（NonConsolidated）決算、予想の修正（ForecastRevision）の開示等があることがわかる。

In [ ]:
# 会計期間の種類を表示
df_fins["TypeOfCurrentPeriod"].value_counts()


In [ ]:
# 事業年度の終了日を表示
df_fins_onlyFY = df_fins[df_fins["TypeOfDocument"].str.contains(
    "FYFinancialStatements")]
df_fins_onlyFY["CurrentPeriodEndDate"].value_counts()


In [ ]:
# 一番多い決算期末の日付でフィルタ
df_fins_onlyFY_onMar = df_fins_onlyFY[df_fins_onlyFY["CurrentPeriodEndDate"] ==
                                      df_fins_onlyFY["CurrentPeriodEndDate"].max().strftime(format("%Y%m%d"))]

# 開示日でグループ化し、開示件数を描画
df_disclosed_date = df_fins_onlyFY_onMar.groupby(
    "DisclosedDate").size().to_frame("Count").reset_index()
px.bar(df_disclosed_date, x="DisclosedDate", y="Count", title="日ごとの開示件数")


ある日に開示が集中していることがわかる（決算期末を迎えてから45日以内に開示しなければならないため）

# マーケット情報 API（毎週第4営業日に週次で更新） **（Lightプラン以上のプランの方のみ利用可能）**

2017年1月から足元までの投資部門別売買状況（金額）のデータを取得することができます。  
なお、配信データは[こちらのページ](https://www.jpx.co.jp/markets/statistics-equities/investor-type/index.html)で公表している内容と同一です。  

In [ ]:
df_trades_spec = cli.get_markets_trades_spec()

# データフレームを確認
df_trades_spec.info()

print("\n")

# データフレームを描画
df_trades_spec


In [ ]:
# 欠損値の確認
df_trades_spec[df_trades_spec.isnull().any(axis=1)]


## 若干のデータ分析

In [ ]:
# 2022年4月以降のプライム市場の投資部門別データを取得
df_trades_spec = cli.get_markets_trades_spec(section=jquantsapi.MARKET_API_SECTIONS.TSEPrime,
                                             from_yyyymmdd=20220408)

# 各属性のうち、Balance（売り買いの差し引き金額）の項目のみ取得
columns_balance = df_trades_spec.columns[df_trades_spec.columns.str.contains(
    "Balance")]
df_trades_spec_only_balance = df_trades_spec[columns_balance].copy()

# 投資部門別データの公表日を選択した項目のデータフレームへ転記
df_trades_spec_only_balance.loc[:, "StartDate"] = df_trades_spec[
    "StartDate"].copy()
df_trades_spec_only_balance.set_index("StartDate", inplace=True)

# 海外投資家、個人、投資信託、信託銀行の4つの属性のみ描画
fig = px.bar(df_trades_spec_only_balance,
             x=df_trades_spec_only_balance.index,
             y=[
                 "ForeignersBalance", "IndividualsBalance",
                 "InvestmentTrustsBalance", "TrustBanksBalance"
             ],
             barmode="group")
fig.update_xaxes(type="category")
fig.show()


In [ ]:
# 日ごとだと見にくいため、月単位で合計してみる
df_monthly = df_trades_spec_only_balance.resample("M").sum()
fig = px.bar(df_monthly,
       x=df_monthly.index,
       y=[
           "ForeignersBalance", "IndividualsBalance",
           "InvestmentTrustsBalance", "TrustBanksBalance"
       ],
       barmode="group")
fig.update_xaxes(
    dtick="M1",
    tickformat="%b %Y",
    tick0="2022-04-30")
fig.show()


2022年4月に海外投資家が大きく買い越しているが、5月以降はほぼ売り越しとなっていそう。（10月から買い戻しの動き？？  11/25追記）
- 2022年4月以前の状況はどうか？
- 今回はプライム市場でフィルタしているが、他の市場区分だとどうか？

# TOPIX指数API **（Lightプラン以上のプランの方のみ利用可能）**

2021年1月から足元までのTOPIX指数のデータを取得することができます。  

In [ ]:
df_topix = cli.get_indices_topix()

# データフレームを確認
df_topix.info()

print("\n")

# データフレームを描画
df_topix


In [ ]:
# 欠損値の確認
df_topix[df_topix.isnull().any(axis=1)]


## 若干のデータ分析（JPXとTOPIX指数の比較）

In [ ]:
# JPXの2021年からの株価を取得
d_now = datetime.now().strftime("%Y%m%d")
df_jpx = cli.get_prices_daily_quotes(code=8697, from_yyyymmdd=20210101, to_yyyymmdd=d_now)
df_topix = cli.get_indices_topix(from_yyyymmdd=20210101, to_yyyymmdd=d_now)

# グラフに第2軸を設定
fig = make_subplots(specs=[[{"secondary_y": True}]])

# ローソク足を描画
fig.add_trace(
    go.Candlestick(x=df_jpx.Date,
                   open=df_jpx.AdjustmentOpen,
                   high=df_jpx.AdjustmentHigh,
                   low=df_jpx.AdjustmentLow,
                   close=df_jpx.AdjustmentClose,
                   name="OHLC"))

# 取引高を第2軸に設定
fig.add_trace(go.Scatter(x=df_topix.Date, y=df_topix.Close, name="TOPIX指数終値", line=dict(color="#51848E")),
              secondary_y=True)
fig.update_layout(title_text="JPX（8697）のローソク足及びTOPIX指数")
fig.show()


# FinVizのTreeMapみたいなやつ

- 各APIを統合して利用した分析として、FinVizのTreeMapのようなものを作ってみる

## 33業種で作成

### 時価総額を算出するため、各銘柄の期末発行済み株式数を取得

In [ ]:
# 2023年初来の財務情報取得
df_fins = cli.get_statements_range(start_dt="20230101")


# 財務情報データから取得したい項目（株式銘柄コード、期末発行済株式数）を定義
fins_columns = [
    "LocalCode",
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"
]

# 財務情報テータから必要なデータのみ取得
df_treasury_stock = df_fins[df_fins[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]
                            != ""][fins_columns]

# 列名の変更（LocalCode -> Code）
df_treasury_stock = df_treasury_stock.rename(columns={"LocalCode": "Code"})


# 期末発行済株式数を文字列から整数に変換
df_treasury_stock[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"] = df_treasury_stock[
        "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"].astype("int")

# 期末発行済株式数を銘柄毎にグループ化して平均
df_treasury_stock = df_treasury_stock.groupby(
    by="Code").mean().reset_index()


### 33業種のそれぞれについて、時価総額Top5の企業を取得

In [ ]:
# 昨日の全銘柄株価を取得
d_yesterday = (datetime.now() - timedelta(days=1)).strftime(format="%Y%m%d")
df_prices = cli.get_prices_daily_quotes(date_yyyymmdd=d_yesterday)

# 銘柄一覧を取得
df_info = cli.get_list()

# 銘柄一覧と全銘柄株価を株式銘柄コードで結合
df_merge = pd.merge(df_prices, df_info, how="inner", on="Code")

# さらに先程取得した期末発行済み株式数と結合
df_merge = pd.merge(df_merge, df_treasury_stock, how="inner", on="Code")

# 時価総額をデータフレームに追加
df_merge["Capitalization"] = df_merge["AdjustmentClose"] * df_merge[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]

# 時価総額及び業種コードでソート
df_merge = df_merge.sort_values(by=["Capitalization"], ascending=False)

# 業種名毎の上位５社を抽出
df_top5 = df_merge.groupby("Sector33CodeName").head(5)

# 業種その他（ETF等）を除外
df_top5 = df_top5[df_top5["Sector33Code"] != "9999"]

# 各業種の時価総額Top5の株式銘柄コードを取得
top5_codes = df_top5["Code"]


### 33業種のそれぞれについて、時価総額Top5の企業の約1か月（20営業日）のリターンを取得

In [ ]:
# ３月の約１か月分の全銘柄の株価情報を取得

df_1month_prices = cli.get_price_range(start_dt="20230228", end_dt="20230331") 

# 各業種の時価総額Top5のみの調整済み終値を取得
df_top5_1month_prices = df_1month_prices[df_1month_prices["Code"].isin(
    top5_codes)][["Code", "Date", "AdjustmentClose"]]

# 約1か月のリターン計算する関数
def calc_return(df, code):
    df = df[df["Code"] == code]
    df["Return"] = df["AdjustmentClose"].pct_change(20)
    return df[["Code", "Return"]].tail(1).to_dict(orient="records")

# 各銘柄の約1か月のリターンを作成
returns_list = []

for code in top5_codes:
    df = calc_return(df_top5_1month_prices, code)
    returns_list.extend(df)

# リターンのデータフレームを作成
df_returns = pd.DataFrame(returns_list)

### 業種、企業名、時価総額、リターンからtreemapを描画

In [ ]:
# リターンのデータフレームを結合
df_top5_with_returns = pd.merge(df_top5, df_returns, how="inner", on="Code")

# treemapを描画
fig = px.treemap(df_top5_with_returns,
                 path=[px.Constant("TSE"), "Sector33CodeName", "CompanyName"],
                 values="Capitalization",
                 color="Return",
                 color_continuous_scale=["red", "white", "green"],
                 color_continuous_midpoint=0)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()


## 17業種でも作成

### 時価総額を算出するため、各銘柄の期末発行済み株式数を取得

In [ ]:
# 2023年初来の財務情報取得
df_fins = cli.get_statements_range(start_dt="20230101")


# 財務情報データから取得したい項目（株式銘柄コード、期末発行済株式数）を定義
fins_columns = [
    "LocalCode",
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"
]

# 財務情報テータから必要なデータのみ取得
df_treasury_stock = df_fins[df_fins[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]
                            != ""][fins_columns]

# 列名の変更（LocalCode -> Code）
df_treasury_stock = df_treasury_stock.rename(columns={"LocalCode": "Code"})


# 期末発行済株式数を文字列から整数に変換
df_treasury_stock[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"] = df_treasury_stock[
        "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"].astype("int")

# 期末発行済株式数を銘柄毎にグループ化して平均
df_treasury_stock = df_treasury_stock.groupby(
    by="Code").mean().reset_index()


### 33業種のそれぞれについて、時価総額Top5の企業を取得

In [ ]:
# 昨日の全銘柄株価を取得
d_yesterday = (datetime.now() - timedelta(days=1)).strftime(format="%Y%m%d")
df_prices = cli.get_prices_daily_quotes(date_yyyymmdd=d_yesterday)


# 銘柄一覧を取得
df_info = cli.get_list()

# 銘柄一覧と全銘柄株価を株式銘柄コードで結合
df_merge = pd.merge(df_prices, df_info, how="inner", on="Code")

# さらに先程取得した期末発行済み株式数と結合
df_merge = pd.merge(df_merge, df_treasury_stock, how="inner", on="Code")

# 時価総額をデータフレームに追加
df_merge["Capitalization"] = df_merge["AdjustmentClose"] * df_merge[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]

# 時価総額及び業種コードでソート
df_merge = df_merge.sort_values(by=["Capitalization"], ascending=False)

# 業種名毎の上位５社を抽出
df_top5 = df_merge.groupby("Sector17CodeName").head(5)

# 業種その他（ETF等）を除外
df_top5 = df_top5[df_top5["Sector17Code"] != "99"]

# 各業種の時価総額Top5の株式銘柄コードを取得
top5_codes = df_top5["Code"]


### 33業種のそれぞれについて、時価総額Top5の企業の約1か月（20営業日）のリターンを取得

In [ ]:
# 9月末から10月末までの約１か月分の全銘柄の株価情報を取得
df_1month_prices = cli.get_price_range(start_dt="20230228", end_dt="20230331") 

# 各業種の時価総額Top5のみの調整済み終値を取得
df_top5_1month_prices = df_1month_prices[df_1month_prices["Code"].isin(
    top5_codes)][["Code", "Date", "AdjustmentClose"]]

# 約1か月のリターン計算する関数
def calc_return(df, code):
    df = df[df["Code"] == code]
    df["Return"] = df["AdjustmentClose"].pct_change(20)
    return df[["Code", "Return"]].tail(1).to_dict(orient="records")

# 各銘柄の約1か月のリターンを作成
returns_list = []

for code in top5_codes:
    df = calc_return(df_top5_1month_prices, code)
    returns_list.extend(df)

# リターンのデータフレームを作成
df_returns = pd.DataFrame(returns_list)

### 業種、企業名、時価総額、リターンからtreemapを描画

In [ ]:
# リターンのデータフレームを結合
df_top5_with_returns = pd.merge(df_top5, df_returns, how="inner", on="Code")

# treemapを描画
fig = px.treemap(df_top5_with_returns,
                 path=[px.Constant("TSE"), "Sector17CodeName", "CompanyName"],
                 values="Capitalization",
                 color="Return",
                 color_continuous_scale=["red", "white", "green"],
                 color_continuous_midpoint=0)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()


# ご意見等
その他、こんなデータが欲しいというご要望や、こんなデータありますか？といった質問もどしどし受け付けております！！

[マケデコdiscord](https://discord.gg/xg6P5RXwaa)の`#j-quants-api`チャンネルに送ってください！！  
よろしくお願いいたします。